In [1]:
# turn off pretty printing, because the slides can't handle the verticality
%pprint

Pretty printing has been turned OFF


In [2]:
# load supriya's ipython extension to capture audio/graphs
%load_ext supriya.ext.ipython

In [3]:
# kill any running scsynth/supernova servers
from supriya.scsynth import kill

kill()

In [4]:
# turn soundcheck on
import supriya

server = supriya.Server().boot()
with server.at():
    with server.add_synthdefs(supriya.default):
        server.add_synth(supriya.default)

In [5]:
# turn soundcheck off
_ = server.quit()

March 14th, 2025

# Supriya: a Python API for SuperCollider

**Joséphine Wolf Oberholtzer (she/her)** <br/>
https://josephine-wolf-oberholtzer.com/

https://github.com/supriya-project/supriya/ <br/>
tree/main/docs/notebooks/supercollider-symposium-2025/presentation.ipynb

<div style="display: flex; flex-direction: row; justify-content: space-around;">
  <div style="text-align: center;">
    <img src="qr-bio.png" height="250" width="250"/><br />
    BIO
  </div>
  <div><img src="apsara.jpg" width="250"/></div>
  <div style="text-align: center;">
    <img src="qr-github.png" height="250" width="250"/><br />
    GITHUB
  </div>
</div>

## Preamble

### What Supriya is for

- A foundational layer for application code
- Consistent API for realtime and non-realtime
- Consistent API for threaded and async concurrency models
- Musical-time-aware clocks
- Especially suited for headless applications
- Let's just make it easy to use SuperCollider inside the Python ecosystem!

### What Supriya isn't for

- Recreating sclang's class library
- Live coding (but you can try... we're gonna do it in a moment)
- UIs (other libraries exist)
- IDEs (other libraries exist)
- MIDI (other libraries exist)
- etc.

### Why Python?

- Many orders of magnitude more people use Python than sclang
- Visibility / support is very good
- Huge vibrant ecosystem (most things I might want somebody already invented)
  - strong support for web frameworks, data science / ml / scientific computing, etc.
- Relatively simple syntax, _initially_ at least
- Good developer experience (tracebacks, debugging, testing, linting, formatting, type checking, CI, docs tooling, etc.)

### What I'm _not_ gonna cover

- How to use Python (although you'll probably learn a little)
- How to use SuperCollider
- How to install Supriya
   - We don't have time!
   - Ask me after!
   - https://supriya-project.github.io/supriya/installation.html
   - `pip install supriya`
- Making really beautiful noises: this is orthogonal

### OK, but what _am_ I gonna cover

- Basic usage
- Design principles
- Server, scores, context entities, synthdefs, osc, clocks, patterns, asyncio, testing, etc.
  - Probably not getting to all of these, but we'll try 

### Design principles?

- **Make it explicit**: avoid globals, avoid implicitness
- **Make it ~boring~ simple**: avoid multiple means to the same ends, resist flexibility, "there should be one (and preferably only one) obvious way to do it"
- **Make it verbose**: give everything (non-fanciful) names, avoid abbreviations, prefer keywords over positionals, strive for using terms of art aligned with the wider language ecosystem
- **Make it self-similar**: strive for identical means of interacting with similar things
- **Make it narrowly focused**: don't have to _implement_ what's already implemented in the rest of the ecosystem, but may have to _integrate_ with it
- **Make it easily introspectable, easily testable**

## Realtime contexts: Servers

### Import supriya

In [ ]:
# supriya is a package, so let's import it
import supriya

In [ ]:
# everything in python is an object, so we can do some inspecting
supriya

In [ ]:
# let's look at the names defined inside the supriya namespace
dir(supriya)

### Import `Server`

In [ ]:
# we can import individual names out of supriya's namespace
from supriya import Server

In [ ]:
# instantiate the server
server = Server()

In [ ]:
# let's look at the names inside the Server namespace
dir(server)

In [ ]:
# actually let's look at the public name inside the Server namespace...
# this is a list comprehension with a filtering clause:
[name for name in dir(server) if not name.startswith("_")]

### Server options

In [ ]:
# print the server's "interpreter representation"
server

In [ ]:
# inspect the server's options
server.options

In [ ]:
# options correspond to CLI flags
!scsynth -h

### Boot the server

In [ ]:
# boot her, returning the server (making this chainable)
server.boot()

### Query the server

In [ ]:
# ask for the query's status
# as reported via /status & /status.reply
server.status

In [ ]:
# print the node tree, assigning to a variable along the way
print(tree := server.query_tree())

In [ ]:
# this is actually a query tree object, not just a string
# ... which is helpful in more complex unit testing situations
# ... because the tree can be annotated with information beyond what scsynth provides
# ... but we can still use string comparisons
tree

### Quit the server

In [ ]:
# quit the server
# note the status in the repr
server.quit()

### Boot with options

In [ ]:
# recall all the options from before
server.options

In [ ]:
# we can use those keywords to configure new options when booting, rebooting, quitting, etc.
server.boot(maximum_logins=2)

### Multiple users

In [ ]:
# We can create a handle to a second server proxy,
# pointed back at the same IP address and port as the first
other_server = Server()
other_server

In [ ]:
# note the port is basically the same as `other_server`
# not counting the -l flag which the connecting server ignores
server

In [ ]:
# connect to the original server via .connect()
other_server.connect()

In [ ]:
# note that the client IDs are different, as expected
print(f"{server.client_id=}")
print(f"{other_server.client_id=}")

In [ ]:
# disconnect from the original server
other_server.disconnect()

In [ ]:
# the original server remains online
server

### Lifecycle events

In [ ]:
# servers emit a variety of "lifecycle events"
# while booting, connecting, disconnecting, quitting and crashing
for event_type in supriya.ServerLifecycleEvent:
    print(repr(event_type))

In [ ]:
# define a simple callback to print the event
def on_event(event):
    print(repr(event))

In [ ]:
# register the callback for every event type
# this is akin to sclang's doWhenBooted
for event_type in supriya.ServerLifecycleEvent:
    server.register_lifecycle_callback(event_type, on_event)
    other_server.register_lifecycle_callback(event_type, on_event)

In [ ]:
# this will go through quitting then booting
server.reboot()

In [ ]:
# this will panic!
other_server.boot()

In [ ]:
# and now just the quitting events
server.quit()

## Context Entities

... entities that live inside a synthesis _context_...

... Groups, Synths, Busses and Buffers ...

### Groups

In [ ]:
server = Server().boot()

In [ ]:
# add a group
(group := server.add_group())

In [ ]:
print(dir(group))

In [ ]:
# verify the group's in the node tree
print(server.query_tree())

In [ ]:
# add a group to the group
child_group = group.add_group()
print(server.query_tree())

In [ ]:
# move the child group into the default group
child_group.move(target_node=server.default_group, add_action="ADD_TO_TAIL")
print(server.query_tree())

In [ ]:
# free the original parent group
group.free()
print(server.query_tree())

### Synths

In [ ]:
# add a synth (this will fail with a warning)
synth = child_group.add_synth(synthdef=supriya.default)

In [ ]:
# yet, we have a proxy to a synth (useless now, i know)
synth

In [ ]:
# but nothing on the server, because the request failed
print(server.query_tree())

#### Completions

In [ ]:
# let's allocate the synthdef and the synth properly
# we use a "moment" to populate a _potential_ bundle
# and a "completion" to populate /d_recv's completion message
with server.at() as moment:
    with server.add_synthdefs(supriya.default) as completion:
        synth = child_group.add_synth(synthdef=supriya.default, frequency=666)

In [ ]:
# wait for the synthdef to load via .sync() then print the node tree
server.sync()
print(server.query_tree())

In [ ]:
# free the synth, just like the group
synth.free()

In [ ]:
# wait, what was that "moment" thing...
moment

In [ ]:
# and what was that "completion" thing...
completion

In [ ]:
# since the full moment is hard to read, let's dig into the requests
moment.requests[0][0]

In [ ]:
# and for the completion too...
completion.requests[0][0]

### Buses

In [ ]:
# add a control bus
(control_bus := server.add_bus())

In [ ]:
# add an audio bus
(audio_bus := server.add_bus("audio"))

In [ ]:
# add a bus group
(control_bus_group := server.add_bus_group(count=4))

In [ ]:
# bus groups don't actually have a concrete reality server-side
# a bus group aggregates together bus objects
for bus in control_bus_group:
    print(bus)

In [ ]:
server.audio_input_bus_group

In [ ]:
server.audio_output_bus_group

#### Shared Memory

In [ ]:
# supriya supports the shared memory interface for control buses
# this works on osx and linux, but not yet on windows
server.shared_memory

In [ ]:
# we can index by an integer, slice, bus or bus group
print(server.shared_memory[0])
print(server.shared_memory[2:8])
print(server.shared_memory[control_bus])
print(server.shared_memory[control_bus_group])

In [ ]:
# and can set bus values directly indexed the same way
server.shared_memory[control_bus_group] = [0.1, 0.5, 0.3, 0.2]
server.shared_memory[control_bus_group]

#### Scopes

### Buffers

In [ ]:
# add a mono buffer with 64 frames
buffer = server.add_buffer(channel_count=1, frame_count=64)

In [ ]:
# query the buffer
buffer.query()

In [ ]:
# generate a chebyshev polynomial in wavetable format
buffer.generate("cheby", amplitudes=[0.1, 0.2, 0.05], as_wavetable=True)

In [ ]:
# get values at indices in the buffer
buffer.get(0, 2, 4, 8)

In [ ]:
# get a range of values
buffer.get_range(index=0, count=16)

In [ ]:
# import plot rendering helper
from supriya import plot

In [ ]:
# plot the buffer (this takes a moment the first time)
plot(buffer)

In [ ]:
# read a file into a buffer
file_path = supriya.samples_path / "birds-01.wav"
other_buffer = server.add_buffer(file_path=file_path)
plot(other_buffer)

In [ ]:
# import play helper
from supriya import play

In [ ]:
# normally play() isn't async
# but for fussy reasons related to jupyter itself being async,
# we have to await here
await play(other_buffer)

In [ ]:
# allocate a group of buffers, e.g.
buffer_group = server.add_buffer_group(count=4, frame_count=512, channel_count=1)
print(buffer_group)

In [ ]:
# buffer groups don't actually have a concrete reality server-side
# but we can iterate over the buffers in the group
for buffer_ in buffer_group:
    print(buffer_)

In [ ]:
# free all the buffers
buffer.free()
other_buffer.free()
buffer_group.free()

## Non-realtime contexts: Scores

In [ ]:
# import and instantiate a Score
from supriya import Score

(score := Score())

In [ ]:
# inspect the score's namespace
# note: no queries, only mutations
[name for name in dir(score) if not name.startswith("_")]

In [ ]:
# add a synthdef at timestamp 0
with score.at(0):
    score.add_synthdefs(supriya.default)

# strum a series of synths
synths = []
for i in range(12):
    with score.at(i / 4):
        frequency = 111 * (i + 1)
        synth = score.add_synth(synthdef=supriya.default, frequency=frequency)
        synths.append(synth)

# free all of them
with score.at(4):
    for synth in synths:
        synth.free()

# pad out a no-op while the envelopes decay
with score.at(5):
    score.do_nothing()

In [ ]:
# play the score (and capture into the notebook)
from supriya import play

_ = await play(score)

In [ ]:
# or just render the score to disk, returning the path and exit code
await score.render()

In [ ]:
# this will error!
# no queries, only mutations
with score.at(0):
    score.query_tree()

In [ ]:
# iterate the osc bundles in the score
for bundle in score.iterate_osc_bundles():
    print(repr(bundle))

In [ ]:
# but actually we store an intermediate format... requests
for bundle in score.iterate_request_bundles():
    print(bundle)

## OSC

### OSC messages & bundles

In [ ]:
from supriya import OscBundle, OscMessage

In [ ]:
message = OscMessage("/this", "is", "a", "message?", 1, 2.5, False)

In [ ]:
# print the interpretation representation
print(repr(message))

In [ ]:
# print the sclang-style hex representation
print(message)

In [ ]:
bundle = OscBundle(timestamp=666.23, contents=[message, message])

In [ ]:
# print the interpretation representation
print(repr(bundle))

In [ ]:
# print the sclang-style hex representation
print(bundle)

### Sending messages

In [ ]:
server.send(["/g_queryTree", 0])

### OSC Callbacks

In [ ]:
captured_messages = []


def procedure(message):
    captured_messages.append(message)


print(
    callback := server.register_osc_callback(
        procedure=procedure, pattern=["/g_queryTree.reply"]
    )
)

In [ ]:
server.send(["/g_queryTree", 0])

In [ ]:
captured_messages

In [ ]:
server.unregister_osc_callback(callback)

### Capturing IO

In [ ]:
# OSC is actually handled by a "protocol" class
server.osc_protocol

In [ ]:
with server.osc_protocol.capture() as transcript:
    with server.at() as moment:
        group = server.add_group()
        subgroup = group.add_group()
        with server.add_synthdefs(supriya.default) as completion:
            synth = subgroup.add_synth(synthdef=supriya.default, frequency=666)
    server.sync()

In [ ]:
for entry in transcript:
    print(entry)

In [ ]:
# the raw message is something else... an intermediate format
transcript[0].raw_message

In [ ]:
moment

In [ ]:
completion

In [ ]:
server.quit()

## Asyncio

from https://docs.python.org/3/library/asyncio.html:

> asyncio is a library to write concurrent code using the async/await syntax.

> asyncio is used as a foundation for multiple Python asynchronous frameworks that provide high-performance network and web-servers, database connection libraries, distributed task queues, etc.

> asyncio is often a perfect fit for IO-bound and high-level structured network code.

In [ ]:
from supriya import AsyncServer

async_server = AsyncServer()

In [ ]:
async def on_boot(event):
    with async_server.at():
        group = async_server.add_group()
        with async_server.add_synthdefs(supriya.default):
            _ = group.add_synth(synthdef=supriya.default, frequency=123)
    await async_server.sync()

In [ ]:
async_server.register_lifecycle_callback("booted", on_boot)

In [ ]:
# start to boot the server, using a random free port
from supriya import find_free_port

# the result of this is actually a coroutine: nothing has really run yet
(coro := async_server.boot(port=find_free_port()))

In [ ]:
# once we await the coroutine, we're golden
await coro

In [ ]:
# async servers require async queries
print(await async_server.query_tree())

In [ ]:
# and now we can quit (asynchronously of course)
await async_server.quit()

In [ ]:
# there are async clocks too!
from supriya import AsyncClock, AsyncOfflineClock, OfflineClock  # noqa: F401

## Contexts

- Mutation interface (all contexts)
- Query interface (realtime only)
- Mutations are realtime/nonrealtime agnostic
- Mutations are concurrency agnostic (".send()" is _always_ sync)
- We can write code targeted against "Context" regardless of what kind

## Synthdefs

### Building SynthDefs (1)

In [ ]:
r"""
SynthDef(\simple, { arg out=0, freq=440;
    Out.ar(out, SinOsc.ar(freq));
});
"""

In [ ]:
# A simple SynthDef using the builder pattern
from supriya.ugens import Out, SinOsc, SynthDefBuilder

with SynthDefBuilder(freq=440, out=0) as builder:
    source = SinOsc.ar(frequency=builder["freq"])
    Out.ar(bus=builder["out"], source=source)

(simple_synthdef := builder.build(name="simple"))

### Graphing SynthDefs

In [ ]:
# a YAML-like textual representation
# this is useful for unit tests!
print(simple_synthdef)

In [ ]:
# a GraphViz representation
from supriya import graph

_ = graph(simple_synthdef)

### SynthDef miscellany

In [ ]:
dir(simple_synthdef)

In [ ]:
simple_synthdef.name

In [ ]:
simple_synthdef.anonymous_name

In [ ]:
simple_synthdef.effective_name

In [ ]:
simple_synthdef.parameters

In [ ]:
simple_synthdef.ugens

In [ ]:
simple_synthdef.has_gate

### Building SynthDefs (2)

In [ ]:
# this is the "default" synthdef as implemented in sclang
r"""
*makeDefaultSynthDef {
    SynthDef(\default, { arg out=0, freq=440, amp=0.1, pan=0, gate=1;
        var z;
        z = LPF.ar(
            Mix.new(VarSaw.ar(freq + [0, Rand(-0.4,0.0), Rand(0.0,0.4)], 0, 0.3, 0.3)),
            XLine.kr(Rand(4000,5000), Rand(2500,3200), 1)
        ) * Linen.kr(gate, 0.01, 0.7, 0.3, 2);
        OffsetOut.ar(out, Pan2.ar(z, pan, amp));
    }, [\ir]).add;
}
"""

In [ ]:
# more imports
from supriya.enums import DoneAction, ParameterRate
from supriya.ugens import (
    LPF,
    Linen,
    Mix,
    OffsetOut,
    Pan2,
    Parameter,
    Rand,
    SynthDefBuilder,
    VarSaw,
    XLine,
)

In [ ]:
# define a builder
builder = SynthDefBuilder(
    out=Parameter(rate=ParameterRate.SCALAR, value=0),
    amplitude=0.1,
    frequency=440,
    gate=1,
    pan=0.5,
)

In [ ]:
# use the builder as a context manager
with builder:
    linen = Linen.kr(
        attack_time=0.01,
        done_action=DoneAction.FREE_SYNTH,
        gate=builder["gate"],
        release_time=0.3,
        sustain_level=0.7,
    )

In [ ]:
# use the builder again
with builder:
    low_pass = LPF.ar(
        source=Mix.new(
            VarSaw.ar(
                frequency=builder["frequency"]
                + (
                    0,
                    Rand.ir(minimum=-0.4, maximum=0.0),
                    Rand.ir(minimum=0.0, maximum=0.4),
                ),
                width=0.3,
            )
        )
        * 0.3,
        frequency=XLine.kr(
            start=Rand.ir(minimum=4000, maximum=5000),
            stop=Rand.ir(minimum=2500, maximum=3200),
        ),
    )

In [ ]:
# and again
with builder:
    panner = Pan2.ar(
        source=low_pass * linen * builder["amplitude"], position=builder["pan"]
    )

In [ ]:
# and again and again
with builder:
    OffsetOut.ar(bus=builder["out"], source=panner)

In [ ]:
(default := builder.build(name="default"))

In [ ]:
_ = graph(default)

### The `synthdef` decorator

In [ ]:
# n.b. I'm not fond of this one because of
# a) how magical it is (not very, but just enough) but mainly because
# b) it makes type-checking difficult
# why difficult? the types of the keyword arguments aren't the same
# as the types of the values as they actually appear inside the function
# when executed at runtime. i can manage this with a mypy plugin,
# but that's just more work for me.
from supriya.ugens import synthdef

In [ ]:
@synthdef("ir")
def default_decorated(out=0, amplitude=0.1, frequency=440, gate=1, pan=0.5):
    linen = Linen.kr(
        attack_time=0.01,
        done_action=DoneAction.FREE_SYNTH,
        gate=gate,
        release_time=0.3,
        sustain_level=0.7,
    )
    low_pass = LPF.ar(
        source=Mix.new(
            VarSaw.ar(
                frequency=frequency
                + (
                    0,
                    Rand.ir(minimum=-0.4, maximum=0.0),
                    Rand.ir(minimum=0.0, maximum=0.4),
                ),
                width=0.3,
            )
        )
        * 0.3,
        frequency=XLine.kr(
            start=Rand.ir(minimum=4000, maximum=5000),
            stop=Rand.ir(minimum=2500, maximum=3200),
        ),
    )
    panner = Pan2.ar(source=low_pass * linen * amplitude, position=pan)
    _ = OffsetOut.ar(bus=out, source=panner)

In [ ]:
default_decorated

In [ ]:
# why joséphine hates @synthdef()...
# the parameters in the decorated signature
# are not the same time as when executed at runtime
from supriya.ugens import Out, SinOsc


@synthdef()
def foo(out=0):
    print(f"out isn't an integer, it's actually {out!r}")
    _ = Out.ar(source=SinOsc.kr())

### UGen methods

In [ ]:
dir(SinOsc)

### SynthDef (de)compilation

In [ ]:
# SynthDefs compile to byte strings
(compiled := default.compile())

In [ ]:
# valid byte strings can be decompiled back into SynthDefs
from supriya.ugens import decompile_synthdef

(decompiled := decompile_synthdef(compiled))

In [ ]:
# sanity-check: the decompiled SynthDef is not the same in memory
default is decompiled

### Compiling via sclang

In [ ]:
# Supriya provides utilities for compiling via sclang.
# This is intended for validating its own logic vs sclang (as a reference spec).
from supriya.ugens import SuperColliderSynthDef

sc_synthdef = SuperColliderSynthDef(
    "foo", "Out.ar(0, SinOsc.ar(freq: 420) * SinOsc.ar(freq: 440))"
)
(sc_compiled_synthdef := sc_synthdef.compile())

In [ ]:
# The sclang-derived SynthDef byte string can be decompiled back into a SynthDef.
print(decompile_synthdef(sc_compiled_synthdef))

### UGen metaprogramming

In [ ]:
from supriya.ugens import UGen, param, ugen


# A dupe of SinOsc
@ugen(ar=True, kr=True, is_pure=True)
class AnotherSinOsc(UGen):
    frequency = param(440.0)
    phase = param(0.0)

In [ ]:
AnotherSinOsc.ar()

In [ ]:
AnotherSinOsc.kr()

In [ ]:
# This won't work because ir=True wasn't set
AnotherSinOsc.ir()

In [ ]:
# A dupe of Out
@ugen(ar=True, kr=True, is_output=True, channel_count=0, fixed_channel_count=True)
class AnotherOut(UGen):
    bus = param(0)
    source = param(unexpanded=True)


AnotherOut.ar(source=AnotherSinOsc.ar())

In [ ]:
from supriya.ugens.pv import PV_ChainUGen


# A dupe of PV_BinShift
@ugen(kr=True, is_width_first=True)
class AnotherPV_BinShift(PV_ChainUGen):
    pv_chain = param()
    stretch = param(1.0)
    shift = param(0.0)
    interpolate = param(0)


# This won't work because of missing pv_chain argument
AnotherPV_BinShift.kr()

In [ ]:
help(ugen)

## Clocks

In [ ]:
# import Clock and instantiate one
from supriya import Clock

(clock := Clock())

In [ ]:
# clocks have bpm tempo
clock.beats_per_minute

In [ ]:
# clocks have time signatures!
clock.time_signature

In [ ]:
# define a simple callback
# this will print the "clock context"
# and return a delta of 1/4 (a quarter note)
# and on the 5th invocation will return a null delta, preventing re-scheduling
def clock_callback(context):
    print(context)
    if context.event.invocations == 4:
        return None
    return 0.25

In [ ]:
# schedule the callback to be run immediately
clock.schedule(clock_callback)

In [ ]:
# start the clock
clock.start()

In [ ]:
# stop the clock
clock.stop()

## Patterns

### Sequence patterns

In [ ]:
# let's make a "sequence pattern", akin to Pseq
from supriya.patterns import SequencePattern

sequence_pattern = SequencePattern([111, 150, 180], iterations=2)
sequence_pattern, sequence_pattern.is_infinite

In [ ]:
# patterns are iterable, so we can loop over them
for x in sequence_pattern:
    print(x)

### Random patterns

In [ ]:
# let's make a "random pattern", akin to Pwhite
from supriya.patterns import RandomPattern

random_pattern = RandomPattern(-1.0, 1.0)
random_pattern, random_pattern.is_infinite  # the pattern is infinite by default

In [ ]:
# because this pattern is infinite we need to break at some point
# we'll use the built-in enumerate() to yield an index we can break on
for i, x in enumerate(random_pattern):
    print(i, x)
    if i == 3:
        break

### Pattern math

In [ ]:
# let's make some choice patterns, akin to Prand
from supriya.patterns import ChoicePattern

frequency_pattern = ChoicePattern(
    [440, 555, 666, [333.33, 366.66], 345], iterations=None
) * ChoicePattern([1, 0.5, 2, 0.25, 4], iterations=None)
for i, x in enumerate(frequency_pattern):
    print(i, x)
    if i == 3:
        break

### Event patterns

In [ ]:
# now create an event pattern, akin to Pbind
from supriya.patterns import EventPattern

event_pattern = EventPattern(
    delta=RandomPattern(0.5, 2),
    duration=RandomPattern(0.05, 0.2),
    frequency=frequency_pattern,
    pan=RandomPattern(-1.0, 1.0),
    synthdef=supriya.default,
)
event_pattern, event_pattern.is_infinite

In [ ]:
# event patterns yield events
for i, event in enumerate(event_pattern):
    print(event)
    if i == 3:
        break

### Structural patterns: buses

In [ ]:
# we can build more complex event patterns
# BusPattern (akin to Pbus) will create a private bus, a group, and link synths
from supriya.patterns import BusPattern

bus_isolated_event_pattern = BusPattern(event_pattern, channel_count=2)

In [ ]:
for i, event in enumerate(bus_isolated_event_pattern):
    print(event)
    if i == 3:
        break

### Aside: some effects SynthDefs

In [ ]:
# now let's define a couple synthdefs for fx
from supriya.ugens import (
    HPF,
    AllpassC,
    FreeVerb,
    In,
    LFNoise1,
    Linen,
    LocalIn,
    LocalOut,
    ReplaceOut,
)

In [ ]:
@synthdef()
def delay(out=0, gate=1):
    envelope = Linen.kr(gate=gate, release_time=0.25, done_action=2)
    source = In.ar(bus=out, channel_count=2)
    tap = AllpassC.ar(
        source=LocalIn.ar(channel_count=2),
        maximum_delay_time=1.0,
        delay_time=LFNoise1.kr(frequency=0.05).scale(-1, 1, 0, 1),
    )
    LocalOut.ar(source=HPF.ar(source=source + tap, frequency=1000) * -0.995)
    Out.ar(bus=out, source=tap * envelope)

In [ ]:
_ = graph(delay)

In [ ]:
@synthdef()
def reverb(out=0, gate=1):
    envelope = Linen.kr(gate=gate, release_time=0.25, done_action=2)
    source = In.ar(bus=out, channel_count=2)
    source = FreeVerb.ar(source=source, mix=0.5, damping=0.5, room_size=0.95)
    ReplaceOut.ar(bus=out, source=source * envelope)

In [ ]:
_ = graph(reverb)

### Structural patterns: effects

In [ ]:
# now some Pfx-type stuff
from supriya.patterns import FxPattern

bus_isolated_fx_pattern = BusPattern(
    FxPattern(event_pattern, synthdef=delay),
    channel_count=2,
)

In [ ]:
# note the second CompositeEvent, coming from the FxPattern
for i, event in enumerate(bus_isolated_fx_pattern):
    print(event)
    if i == 3:
        break

### Structural patterns: parallelism

In [ ]:
# now let's do it in parallel, Ppar-style
from supriya.patterns import GroupPattern, ParallelPattern

final_pattern = BusPattern(
    FxPattern(
        pattern=GroupPattern(ParallelPattern([bus_isolated_fx_pattern] * 4)),
        synthdef=reverb,
    ),
    channel_count=2,
)

In [ ]:
for i, event in enumerate(final_pattern):
    print(event)
    if i == 3:
        break

### Pattern players

In [ ]:
# ok, let's play it
def on_boot(event):
    server.add_synthdefs(supriya.default, delay, reverb)
    server.sync()


server = Server()
server.register_lifecycle_callback("booted", on_boot)
server.boot()
clock = Clock()

In [ ]:
(pattern_player := final_pattern.play(context=server, clock=clock))

In [ ]:
print(server.query_tree())

In [ ]:
pattern_player.stop()

In [ ]:
server.quit()

## Cleanliness

- ci/cd
- docs
- testing
- typing

## Future work?

- More docs
- More examples
- DAW affordances
    - multi-context mixers
    - tracks & subtracks
    - send & receives
    - modulation
    - transport

## Ciao ciao! 🙇🏼‍♀️🏳️‍⚧️💓

Questions?

Thanks, darlings! <br/>
xoxo, joséphine

https://josephine-wolf-oberholtzer.com/ <br/>
https://github.com/supriya-project/supriya/

<div style="display: flex; flex-direction: row; justify-content: space-around;">
  <div style="text-align: center;">
    <img src="qr-bio.png" height="250" width="250"/><br />
    BIO
  </div>
  <div><img src="apsara.jpg" width="250"/></div>
  <div style="text-align: center;">
    <img src="qr-github.png" height="250" width="250"/><br />
    GITHUB
  </div>
</div>